In [33]:
drive.flush_and_unmount()

## Setup

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
import matplotlib.pyplot as plt
import numpy as np
import tensorflow as tf
import tensorflow_datasets as tfds
from tensorflow.keras import layers
import os
from PIL import Image

## Data Augmentation Steps

In [3]:
# Public Dataset 

train_images_path = "/content/drive/MyDrive/researchpaper2023/00_datasets/public_dataset/train/images"

train_images = []
for file in os.listdir(train_images_path):
    if file.endswith(".jpg"):
        im = Image.open(train_images_path + "/" + file)
        train_images.append(im)

print("loaded " + str(len(train_images)) + " train images")

test_images_path = "/content/drive/MyDrive/researchpaper2023/00_datasets/public_dataset/test/images"
test_images = []
for file in os.listdir(test_images_path):
    if file.endswith(".jpg"):
        im = Image.open(test_images_path + "/" + file)
        test_images.append(im)

print("loaded " + str(len(test_images)) + " test images")

loaded 26 train images
loaded 12 test images


In [4]:
def augment_data(images):
  augmented_images = []
  augmented_images_rotation = []
  for image in images:
    for i in range(3):
      seed = (i, 0)  # tuple of size (2,)
      stateless_random_contrast = tf.keras.utils.array_to_img(tf.image.stateless_random_contrast(image, lower=0.4, upper=0.6, seed=seed))
      stateless_random_brightness = tf.keras.utils.array_to_img(tf.image.stateless_random_brightness(image, max_delta=0.2, seed=seed))
      stateless_random_hue = tf.keras.utils.array_to_img(tf.image.stateless_random_hue(image, max_delta=0.2, seed=seed))
      stateless_random_saturation = tf.keras.utils.array_to_img(tf.image.stateless_random_saturation(image, lower=0.2, upper=0.8, seed=seed))

      augmented_images.append((stateless_random_contrast, image.filename, "_augmented_contrast_" + str(i)))
      augmented_images.append((stateless_random_brightness, image.filename, "_augmented_brightness_" + str(i)))
      augmented_images.append((stateless_random_hue, image.filename, "_augmented_hue_" + str(i)))
      augmented_images.append((stateless_random_saturation, image.filename, "_augmented_saturation_" + str(i)))

      stateless_90deg_rotation = tf.keras.utils.array_to_img(tf.image.rot90(stateless_random_brightness))
      augmented_images_rotation.append((stateless_90deg_rotation, image.filename, "_augmented_brightness_rotated_" + str(i)))

      stateless_90deg_rotation_hue = tf.keras.utils.array_to_img(tf.image.rot90(stateless_random_hue))
      augmented_images_rotation.append((stateless_90deg_rotation_hue, image.filename, "_augmented_hue_rotated_" + str(i)))

      stateless_90deg_rotation_saturation = tf.keras.utils.array_to_img(tf.image.rot90(stateless_random_saturation))
      augmented_images_rotation.append((stateless_90deg_rotation_saturation, image.filename, "_augmented_saturation_rotated_" + str(i)))
  return (augmented_images, augmented_images_rotation)

(augmented_train_images, augmented_train_images_rotation) = augment_data(train_images)
print("augmented " + str(len(augmented_train_images) + len(augmented_train_images_rotation)) + " images for train set")

(augmented_test_images, augmented_test_images_rotation) = augment_data(test_images)
print("augmented " + str(len(augmented_test_images) + len(augmented_test_images_rotation)) + " images for test set")

augmented 546 images for train set
augmented 252 images for test set


In [9]:
import shutil
base_train_path = os.path.dirname(train_images_path)
base_test_path = os.path.dirname(test_images_path)
os.makedirs(base_train_path + '_augmented/images', exist_ok=True)
os.makedirs(base_train_path + '_augmented/labels', exist_ok=True)
os.makedirs(base_train_path + '_augmented/mask_content', exist_ok=True)
os.makedirs(base_test_path + '_augmented/images', exist_ok=True)
os.makedirs(base_test_path + '_augmented/labels', exist_ok=True)
os.makedirs(base_test_path + '_augmented/mask_content', exist_ok=True)

for (image, filepath, postfix) in (augmented_train_images + augmented_test_images):
  base_path = os.path.dirname(os.path.dirname(filepath))
  image_folder_path = base_path + "_augmented" + "/images/"
  labels_folder_path_from = base_path + "/labels/"
  labels_folder_path_to = base_path + "_augmented" + "/labels/"

  filename = os.path.basename(filepath)
  image.save(image_folder_path + filename[:-4] + postfix + ".jpg")

  labels_path = labels_folder_path_from + filename[:-4] + ".txt"
  augmented_labels_path = labels_folder_path_to + filename[:-4] + postfix + ".txt"
  shutil.copyfile(labels_path,augmented_labels_path)

for (image, filepath, postfix) in (augmented_train_images_rotation + augmented_test_images_rotation):
  base_path = os.path.dirname(os.path.dirname(filepath))
  image_folder_path = base_path + "_augmented" + "/images/"
  labels_folder_path_from = base_path + "/labels/"
  labels_folder_path_to = base_path + "_augmented" + "/labels/"

  filename = os.path.basename(filepath)
  image.save(image_folder_path + filename[:-4] + postfix + ".jpg")

  labels_path = labels_folder_path_from + os.path.basename(filename)[:-4] + ".txt"
  augmented_labels_path = labels_folder_path_to + os.path.basename(filename)[:-4] + postfix + ".txt"
  
  with open(labels_path, 'r') as f:
    lines = f.read().splitlines()
    # for every line in lines, split by space
    polygons = []
    for line in lines:
        l = line.split(" ")[1:]
        # even numbers are x, odd numbers are y, make a tuple of x,y
        points = [(float(l[i]), float(l[i+1])) for i in range(0, len(l), 2)]
        # center each point, to be between -0.5 and 0.5 (they're relative coordinates from 0.0 to 1.0)
        points = [(p[0] - 0.5, p[1] - 0.5) for p in points]
        # rotate 90deg and re-transform to be between 0 and 1.
        points = [(p[1] + 0.5, -p[0] + 0.5) for p in points]
        
        polygons.append(points)
  new_labels = "\n".join(["0 " + " ".join([str(p[0]) + " " + str(p[1]) for p in poly]) for poly in polygons])
  with open(augmented_labels_path, 'w') as f:
    f.write(new_labels)

/content/drive/MyDrive/researchpaper2023/00_datasets/public_dataset/train_augmented/images
/content/drive/MyDrive/researchpaper2023/00_datasets/public_dataset/test_augmented/images


Finished augmenting the public dataset!

# Combine generated images with public domain training dataset

In [12]:
import os
import shutil

def copy_dataset(base_dir, from_dir, to_dir):
  os.makedirs(base_dir + to_dir + "images/", exist_ok=True)
  os.makedirs(base_dir + to_dir + "labels/", exist_ok=True)

  p = base_dir + from_dir + "labels/"
  for filename in os.listdir(p):
    # copy over label file to new combined destination
    label_filename = filename
    label_filepath_from = base_dir + from_dir + "labels/" + label_filename
    label_filepath_to = base_dir + to_dir + "labels/" + label_filename
    shutil.copyfile(label_filepath_from, label_filepath_to)
    
    # copy over image file to new combined destination
    image_filename = filename[:-4] + ".jpg"
    image_filepath_from = base_dir + from_dir + "images/" + image_filename
    image_filepath_to = base_dir + to_dir + "images/" + image_filename
    shutil.copyfile(image_filepath_from, image_filepath_to)

base_dir = "/content/drive/MyDrive/researchpaper2023/00_datasets/"

In [13]:
os.makedirs(base_dir + "combined_train_dataset/", exist_ok=True)

## copy all public_dataset train images and labels to combined dataset folder
copy_dataset(base_dir, from_dir="public_dataset/train/", to_dir="combined_train_dataset/train/")
print("Done copying from public_dataset/train/ to combined_train_dataset/train/")

copy_dataset(base_dir, from_dir="public_dataset/train_augmented/", to_dir="combined_train_dataset/train/")
print("Done copying from public_dataset/train_augmented/ to combined_train_dataset/train/")

copy_dataset(base_dir, from_dir="generated_images/train/", to_dir="combined_train_dataset/train/")
print("Done copying from generated_images/train/ to combined_train_dataset/train/")

Done copying from public_dataset/train/ to combined_train_dataset/train/
Done copying from public_dataset/train_augmented/ to combined_train_dataset/train/
Done copying from generated_images/train/ to combined_train_dataset/train/


In [14]:
# count the number of training images total 
combined_count = 0
p = "/content/drive/MyDrive/researchpaper2023/00_datasets/combined_train_dataset/train/images/"
for filename in os.listdir(p):
  combined_count += 1

# count the number of unaugmented public training images  
public_noaug_count = 0
p = "/content/drive/MyDrive/researchpaper2023/00_datasets/public_dataset/train/labels/"
for filename in os.listdir(p):
  public_noaug_count += 1

# count the number of augmented public training images  
public_aug_count = 0
p = "/content/drive/MyDrive/researchpaper2023/00_datasets/public_dataset/train_augmented/labels/"
for filename in os.listdir(p):
  public_aug_count += 1

# count the number of generated training images  
generated_count = 0
p = "/content/drive/MyDrive/researchpaper2023/00_datasets/generated_images/train/labels/"
for filename in os.listdir(p):
  generated_count += 1

# count the number of augmented public domain test images  
public_augtest_count = 0
p = "/content/drive/MyDrive/researchpaper2023/00_datasets/public_dataset/test_augmented/images"
for filename in os.listdir(p):
  public_augtest_count += 1

print('There are a total of:', combined_count, 'images in the combined train/val dataset')
print(public_noaug_count, ' of the images are from the unaugmented public dataset')
print(public_aug_count, ' of the images are from the augmented public dataset')
print(generated_count, ' of the images are from the generated dataset')
print('\n')
print('There are a total of:', public_augtest_count, 'images in the test dataset (public domain only)')

There are a total of: 1459 images in the combined train/val dataset
26  of the images are from the unaugmented public dataset
546  of the images are from the augmented public dataset
887  of the images are from the generated dataset


There are a total of: 252 images in the test dataset (public domain only)


In [32]:
import os

# for labels_file in os.listdir('/content/drive/MyDrive/researchpaper2023/00_datasets/combined_train_dataset/train/labels'):
#   f = open('/content/drive/MyDrive/researchpaper2023/00_datasets/combined_train_dataset/train/labels/' + labels_file, 'r')
#   lines = f.readlines()
#   for line in lines:
#     if line[0] == "1":
#       print(f)

for labels_file in os.listdir('/content/drive/MyDrive/researchpaper2023/00_datasets/combined_train_dataset/train/labels'):
  f = open('/content/drive/MyDrive/researchpaper2023/00_datasets/combined_train_dataset/train/labels/' + labels_file, 'r')
  lines = f.readlines()
  for line in lines:
    if line[0] == "1":
      print(labels_file)

# Add 1000 Pascal VOC Dataset images to test set 


In [17]:
# https://stackoverflow.com/questions/31434278/how-do-i-use-python-pil-to-save-an-image-to-a-particular-directory 
import urllib.request
from PIL import Image
import tarfile
import os

voc_base_dir = base_dir + "VOCtrainval_11-May-2012"
voc_images_dir = voc_base_dir + '/VOCdevkit/VOC2012/JPEGImages/'
voc_test_subset_dir = base_dir + "VOC_dataset" + '/test/'
voc_train_subset_dir = base_dir + "VOC_dataset" + '/train/'

Don't run the following cell if you already have the dataset. It'll take a long time to run.

In [ ]:
urllib.request.urlretrieve("http://host.robots.ox.ac.uk/pascal/VOC/voc2012/VOCtrainval_11-May-2012.tar", voc_base_dir + ".tar")
# Extract the tar file
tar = tarfile.open(voc_base_dir + ".tar")

# save extracted files to a folder
tar.extractall(path=voc_base_dir)
# close the tar file
tar.close()

In [ ]:
# Copy the first 1000 images from extracted folder to a new folder
# Resizing to 512x512
# Create a full black mask image
# Create an empty labels file
def copy_image_and_make_metadata(voc_subset_dir, all_images_filenames):
  # Make a new directory for the subset
  voc_images_dir_subset_images = voc_subset_dir + "images/"
  voc_images_dir_subset_labels = voc_subset_dir + "labels/"
  voc_images_dir_subset_mask_content = voc_subset_dir + "mask_content/"
  os.makedirs(voc_images_dir_subset_images, exist_ok=True)
  os.makedirs(voc_images_dir_subset_labels, exist_ok=True)
  os.makedirs(voc_images_dir_subset_mask_content, exist_ok=True)

  for (i, filename) in enumerate(all_images_filenames):
    if i % 10 == 0:
      print('images done', i, "/", len(all_images_filenames))
    image_from = voc_images_dir + filename
    image_to = voc_images_dir_subset_images + filename
    image_mask_to = voc_images_dir_subset_mask_content + filename[:-4] + ".png"
    labels_to = voc_images_dir_subset_labels + filename[:-4] + ".txt"

    image = Image.open(image_from)
    # resize image to 512x512
    image = image.resize((512, 512))
    image.save(image_to)
    
    # make a new image of the same size as `image`, fill it with black
    mask_image = Image.new('RGB', image.size, (0, 0, 0))
    # save the mask image
    mask_image.save(image_mask_to)

    open(labels_to, 'w').close()

all_images_filenames_it = os.listdir(voc_images_dir)
all_images_filenames = []
for filename in all_images_filenames_it:
  if len(all_images_filenames) >= 1700:
    break
  all_images_filenames.append(filename)

# print("Creating voc test set")
# copy_image_and_make_metadata(voc_test_subset_dir, all_images_filenames[:1000])

print("Creating voc train set")
copy_image_and_make_metadata(voc_train_subset_dir, all_images_filenames[1000:1700])
  




Creating voc train set
images done 0 / 700
images done 10 / 700
images done 20 / 700
images done 30 / 700
images done 40 / 700
images done 50 / 700
images done 60 / 700
images done 70 / 700
images done 80 / 700
images done 90 / 700
images done 100 / 700
images done 110 / 700
images done 120 / 700
images done 130 / 700
images done 140 / 700
images done 150 / 700
images done 160 / 700
images done 170 / 700
images done 180 / 700
images done 190 / 700
images done 200 / 700
images done 210 / 700
images done 220 / 700
images done 230 / 700
images done 240 / 700
images done 250 / 700
images done 260 / 700
images done 270 / 700
images done 280 / 700
images done 290 / 700
images done 300 / 700
images done 310 / 700
images done 320 / 700
images done 330 / 700
images done 340 / 700
images done 350 / 700
images done 360 / 700
images done 370 / 700
images done 380 / 700
images done 390 / 700
images done 400 / 700
images done 410 / 700
images done 420 / 700
images done 430 / 700
images done 440 / 70

In [30]:
# count the number of images in our VOC subset
count = 0
for filename in os.listdir(voc_test_subset_dir + "images/"):
  count += 1

print('There are a total of:', count, 'in', voc_test_subset_dir)

# count the number of images in our VOC subset
count = 0
for filename in os.listdir(voc_train_subset_dir + "images/"):
  count += 1

print('There are a total of:', count, 'in', voc_train_subset_dir)

There are a total of: 1000 in /content/drive/MyDrive/researchpaper2023/00_datasets/VOC_dataset/test/
There are a total of: 700 in /content/drive/MyDrive/researchpaper2023/00_datasets/VOC_dataset/train/


# For Victoria by Ben
Run the following to copy over the VOC dataset

In [ ]:
os.makedirs('/content/drive/MyDrive/researchpaper2023/00_datasets/testaug_voc_combined/test/images', exist_ok=True)
os.makedirs('/content/drive/MyDrive/researchpaper2023/00_datasets/testaug_voc_combined/test/labels', exist_ok=True)

In [25]:
# copy VOC images to testaug_voc_combined - this is our final test set
import shutil

p = '/content/drive/MyDrive/researchpaper2023/00_datasets/VOC_dataset/test/images/'
for filename in os.listdir(p):
  # copy over label file to new combined destination
  label_filename = filename
  label_filepath_from = p + label_filename
  label_filepath_to = '/content/drive/MyDrive/researchpaper2023/00_datasets/testaug_voc_combined/test/images/' + label_filename
  shutil.copyfile(label_filepath_from, label_filepath_to)

  image_filename = filename[:-4] + ".txt"
  image_filepath_from = os.path.dirname(os.path.dirname(p)) + "/labels/" + image_filename
  image_filepath_to = '/content/drive/MyDrive/researchpaper2023/00_datasets/testaug_voc_combined/test/labels/' + image_filename
  shutil.copyfile(image_filepath_from, image_filepath_to)

In [26]:
# copy public dataset > test_augmented images to testaug_voc_combined - this is our final test set
import shutil

p = '/content/drive/MyDrive/researchpaper2023/00_datasets/public_dataset/test_augmented/images/'
for filename in os.listdir(p):
  # copy over label file to new combined destination
  label_filename = filename
  label_filepath_from = p + label_filename
  label_filepath_to = '/content/drive/MyDrive/researchpaper2023/00_datasets/testaug_voc_combined/test/images/' + label_filename
  shutil.copyfile(label_filepath_from, label_filepath_to)

  image_filename = filename[:-4] + ".txt"
  image_filepath_from = os.path.dirname(os.path.dirname(p)) + "/labels/" + image_filename
  image_filepath_to = '/content/drive/MyDrive/researchpaper2023/00_datasets/testaug_voc_combined/test/labels/' + image_filename
  shutil.copyfile(image_filepath_from, image_filepath_to)

In [28]:
# copy VOC images to combined_train_dataset - this is our final train set
import shutil

p = '/content/drive/MyDrive/researchpaper2023/00_datasets/VOC_dataset/train/images/'
for filename in os.listdir(p):
  # copy over label file to new combined destination
  label_filename = filename
  label_filepath_from = p + label_filename
  label_filepath_to = '/content/drive/MyDrive/researchpaper2023/00_datasets/combined_train_dataset/train/images/' + label_filename
  shutil.copyfile(label_filepath_from, label_filepath_to)

  image_filename = filename[:-4] + ".txt"
  image_filepath_from = os.path.dirname(os.path.dirname(p)) + "/labels/" + image_filename
  image_filepath_to = '/content/drive/MyDrive/researchpaper2023/00_datasets/combined_train_dataset/train/labels/' + image_filename
  shutil.copyfile(image_filepath_from, image_filepath_to)

In [29]:
p = '/content/drive/MyDrive/researchpaper2023/00_datasets/testaug_voc_combined/test/images/'
count = 0
for filename in os.listdir(p):
  count += 1
print(count)

p = '/content/drive/MyDrive/researchpaper2023/00_datasets/testaug_voc_combined/test/labels/'
count = 0
for filename in os.listdir(p):
  count += 1
print(count)

p = '/content/drive/MyDrive/researchpaper2023/00_datasets/combined_train_dataset/train/images/'
count = 0
for filename in os.listdir(p):
  count += 1
print(count)

p = '/content/drive/MyDrive/researchpaper2023/00_datasets/combined_train_dataset/train/labels/'
count = 0
for filename in os.listdir(p):
  count += 1
print(count)

1252
1252
2159
2159
